# Change Constraints

In a stochastic program with chance constraints we make an optimal decision prior to the realization of random data while allowing constraints to be violated with a probability specified by us. Chance constraints are also called probabilistic constraints.


In this post we will give the general mathematical description for stochastic programs with chance constraints and explain the following types of change constraints:

- single chance constraint
- joint chance constraints
- individual chance constraints

Moreover we will implement a basic example of each type in pyomo.

## Mathematical formulation

A stochastic program with chance constraints can be expressed mathematically as follows:

$$
\begin{array}{ll}
\min_x & c\cdot x \\
s.t.   & P(Ax\leq b) \geq p\\
       & x\geq 0
\end{array}
$$

where the symbols are denoting:

- $x\in\mathbb{R}^n$ the decision variable, 
- $c\in\mathbb{R}^n$ the coefficients in the objective function,
- $A\in\mathbb{R}^{m\times n}$ coefficient of the constraints,
- $b\in\mathbb{R}^m$ the right hand side of the constraints and
- $p\in(0,1]$ is the probability that the constraints (or some of them) may be violated.

The distinguish feature of a stochastic program with chance constraint is $P(Ax\le b) \geq p$, which allows that the constraints may be violated with *risk tolerance* $\epsilon=1-p$.

## Scenario based solution of models with chance constraints

Supose we can describe the distribution $P(Ax\leq b)$ by set $S$ of scenarios. Then we can reformulate the stochastic program with change constraints as a mixed-integer problem. Therefore introduce a binary variable $y_k$ which equals 1 if and only if the constraint is satisfied in scenario $k$. Afterwards the stochastic program with chance constraint is given by:

$$
\begin{array}{ll}
\min & c x \\
s.t. & A_kx \leq b_k + M_k(1-y_k) \\
     & \sum_{k\in S} p_k y_k \geq p \\
     & x\geq0,  y\in \{0,1\}^{|S|}
\end{array}
$$

Here $M_k\in\mathbb{R}^m$ is a suitable choosen big-M vector and $p_k$ the likelihood of scenario $k$.

# Single change constraints

The simplest case is when there is only one constraint - a single chance constraint.

The following example is taken from the GAMS documentation:

$$
\begin{array}{llc}
\min & x_1+x_2 &\\
s.t. & p(\omega x_1+x_2 \geq 7) \geq 0.75 & \; \omega\in S\\
     & x_1,x_2\geq 0 &
\end{array}
$$

Suppose we have four equaly likely scenarios

In [ ]:
I = [1,2]
S = [1,2,3,4]
Omega = {1:1, 2:2, 3:3, 4:4}
prob = {1:0.25, 2:0.25, 3:0.25, 4:0.25}
risk = 0.75

In [ ]:
import pyomo.environ as pyo
import pandas as pd

A pyomo implementation is given in the next code cell.

In [ ]:
def single_cc(risk, I, S, prob, omega):
    ## sets
    m = pyo.ConcreteModel()
    m.I = pyo.Set(initialize = I, doc = 'index decision var')
    m.S = pyo.Set(initialize = S, doc = 'scenario set')
    
    ## parameter
    @m.Param(m.S)
    def omega(m,s):
        return Omega[s]
    
    @m.Param(m.S)
    def p(m,s):
        return prob[s]
    
    m.M = pyo.Param(initialize = 1000)
    m.r = pyo.Param(initialize = 1 - risk, doc = 'risk tolerance')
    
    # vars
    m.x = pyo.Var(m.I, domain = pyo.NonNegativeReals)
    m.y = pyo.Var(m.S, domain = pyo.Binary)
    m.cc = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, m.r))
    
    # objective
    m.OBJ = pyo.Objective(expr = sum(m.x[i] for i in m.I), sense = pyo.minimize)
    
    # constraints
    @m.Constraint(m.S)
    def c1(m, s):
        return m.omega[s] * m.x[1] + m.x[2] >= 7 - m.M * (1 - m.y[s])
    
    m.cc_def = pyo.Constraint(expr = m.cc == 1 - sum(m.p[k] * m.y[k] for k in m.S))
    
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return m

### inspect solution

Lets inspect the values of the optimal solution:

In [ ]:
m = single_cc(risk, I,S,prob,Omega)
m.x.pprint()

x : Size=2, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   3.5 :  None : False : False : NonNegativeReals
      2 :     0 :   0.0 :  None : False : False : NonNegativeReals


Lets inspect which scenarios were violated.

In [ ]:
m.y.pprint()

y : Size=4, Index=S
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary


### Evaluating solution

As each scenario has a likelihood of 0.25 and our risk tolerance was 0.25 too, we allow precisely one scenario/constraint to be violated.

Obviously the first scenario/constraint was violated, because here the coefficient of $x_1$ is smallest.

# Multiple chance constraints


## Joint chance constraints

In stochastic programs with joint chance constraints all constraints have to be satisfied simultaneously.

The following example is taken from the GAMS documentation:

$$
\begin{array}{llc}
\min & x_1 + x_2 & \\
s.t. & P(\omega_1 x_1 +x_2 \geq 7, \omega_2 x_1 +3 x_2 \geq 12) \geq 0.6 & (\omega_1,\omega_2) \in S \\
     & x_1,x_2\geq 0 &
\end{array}
$$

Suppose we have 12 equaly likely scenarios.

In [ ]:
#| code-fold: true

S = {
    1: {'omega_1': 1, 'omega_2': 1, 'probability': 1./12. },
    2: {'omega_1': 1, 'omega_2': 2, 'probability': 1./12. },
    3: {'omega_1': 1, 'omega_2': 3, 'probability': 1./12. },
    4: {'omega_1': 2, 'omega_2': 1, 'probability': 1./12. },
    5: {'omega_1': 2, 'omega_2': 2, 'probability': 1./12. },
    6: {'omega_1': 2, 'omega_2': 3, 'probability': 1./12. },
    7: {'omega_1': 3, 'omega_2': 1, 'probability': 1./12. },
    8: {'omega_1': 3, 'omega_2': 2, 'probability': 1./12. },
    9: {'omega_1': 3, 'omega_2': 3, 'probability': 1./12. },
    10: {'omega_1': 4, 'omega_2': 1, 'probability': 1./12. },
    11: {'omega_1': 4, 'omega_2': 2, 'probability': 1./12. },
    12: {'omega_1': 4, 'omega_2': 3, 'probability': 1./12. },
}
p = 0.6
df = pd.DataFrame(S).T
df.index.name = 'sceanrio'
df

,omega_1,omega_2,probability
sceanrio,,,
1,1.0,1.0,0.083333
2,1.0,2.0,0.083333
3,1.0,3.0,0.083333
4,2.0,1.0,0.083333
5,2.0,2.0,0.083333
6,2.0,3.0,0.083333
7,3.0,1.0,0.083333
8,3.0,2.0,0.083333
9,3.0,3.0,0.083333


A pyomo implementation is given in the next code cell.

In [ ]:
def joint_cc(S, p):
    
    m = pyo.ConcreteModel()
    # sets
    m.I = pyo.Set(initialize = [1,2], doc = 'index decision var')
    m.S = pyo.Set(initialize = S.keys(), doc = 'scenario indices')
    
    # param
    m.r = pyo.Param(initialize = 1 - p, doc = '1-p - risk tolerance')
    m.M = pyo.Param(initialize = 100, doc = 'big-M')
    
    @m.Param(m.S, doc ='scenario probability')
    def p(m,s):
        return S[s]['probability']
    
    @m.Param(m.S, doc = 'coefficient omega_1')
    def omega1(m,s):
        return S[s]['omega_1']
    @m.Param(m.S, doc = 'coefficient omega_2')
    def omega2(m,s):
        return S[s]['omega_2']
    
    # vars
    m.x = pyo.Var(m.I, domain = pyo.NonNegativeReals)
    m.y = pyo.Var(m.S, domain = pyo.Binary)
    m.cc = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, m.r))
    
    # objective
    m.OBJ = pyo.Objective(expr = sum(m.x[i] for i in m.I), sense = pyo.minimize)
    
    
    # constraints
    @m.Constraint(m.S, doc = '1st inequal of joint prob')
    def c1(m, s):
        return m.omega1[s] * m.x[1] + m.x[2] >= 7 - m.M * (1- m.y[s])
    
    @m.Constraint(m.S, doc = '2nd inequal of joint prob')
    def c2(m, s):
        return m.omega2[s] * m.x[1] + 3*m.x[2] >= 12 - m.M * (1- m.y[s])
    
    m.cc_def = pyo.Constraint(expr = m.cc == 1 - sum(m.p[k] * m.y[k] for k in m.S))
    
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return(m)

### inspect solution and check implementation

First we inspect the values of the decision variables.

In [ ]:
m = joint_cc(S,p)
# inspect decision
m.x.pprint()

x : Size=2, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.8 :  None : False : False : NonNegativeReals
      2 :     0 :   3.4 :  None : False : False : NonNegativeReals


We look which scenarios were violated next.

In [ ]:
# inspect violation
m.y.pprint()

y : Size=12, Index=S
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   1.0 :     1 : False : False : Binary
      9 :     0 :   1.0 :     1 : False : False : Binary
     10 :     0 :   1.0 :     1 : False : False : Binary
     11 :     0 :   1.0 :     1 : False : False : Binary
     12 :     0 :   1.0 :     1 : False : False : Binary


At last we check the value of the corresponding risk.

In [ ]:
# inspect risk
m.cc.pprint()

cc : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 0.333333333333333 :   0.4 : False : False : NonNegativeReals


### evaluating solution

We see that both constraints are satisfied in 8 scenarios. Indeed as there are 12 equally likely scenarios, at least $12\cdot 0.6 \approx 7.2$ scenarios have to be satisfied.

Obviously the solution violates the scenarios/constraints with the smallest coeffients for $x_1$.

## Individual chance constraints

If there is no correlation between the probabilities of the matrix $A$, then we have a stochastic problem with individual chance constraints and the linear program has the following form:

$$
\begin{array}{llc}
\min & cx & \\
s.t. & P(A_ix \leq b_i) \geq p_i,& \text{for}\; i = 1,\ldots,m\\
     & x \geq 0 &
\end{array}
$$


Again we use an example from the GAMS documentation:

$$
\begin{array}{lll}
\min & x_1 + x_2 & \\
s.t. & P(\omega_1 x_1 + x 2 \geq 7 ) \geq 0.75, & \omega_1\in S_1:=\{1,2,3,4\} \\
     & P(\omega_2 x_1 + 3*x 2 \geq 12 ) \geq 0.6, & \omega_2\in S_2:=\{1,2,3\} \\
     & P(\omega_1 x_1 + \omega_2*x 2 \geq 10 ) \geq 0.5, & \omega_2\in S:= S_1 \times S_2 \\
     & x_1,x_2 \geq 0
\end{array}
$$

- a binary decision variable $y_{s,i}$ for each individual chance constraint $i$ and each scenario $s$
- a "risk variable" $cc_i$ for each individual chance constraint

In [ ]:
# data
S1 = {1:1, 2:2, 3:3, 4:4}
S2 = {1:1, 2:2, 3:3}

A pyomo implementation is given in the next code cell.

In [ ]:
# model not written for speed, e.g. contains redundant constraints
def individual_cc(S1,S2):
    m = pyo.ConcreteModel('Individual Chance Constraints')
    
    #sets
    m.I = pyo.Set(initialize = [1,2], doc = 'index decision variables')
    m.S1 = pyo.Set(initialize = S1.keys(), doc = 'index set S1')
    m.S2 = pyo.Set(initialize = S2.keys(), doc = 'index set S2')
    m.S = pyo.Set(initialize = m.S1 * m.S2, doc = 'index scenarios')
    
    # parameter
    m.M = pyo.Param(initialize = 1000., doc = 'big-M')
    
    @m.Param(m.S, doc ='scenario probability')
    def p(m,_s1,_s2):
        return 1. / len(m.S)
    @m.Param(m.S1, doc = 'coefficients for S1')
    def omega1(m,s1):
        return S1[s1]
    @m.Param(m.S2, doc = 'coefficients for S2')
    def omega2(m,s2):
        return S2[s2]
    
    # vars
    m.x = pyo.Var(m.I, domain = pyo.NonNegativeReals)
    m.y1 = pyo.Var(m.S, domain = pyo.Binary)
    m.y2 = pyo.Var(m.S, domain = pyo.Binary)
    m.y3 = pyo.Var(m.S, domain = pyo.Binary)
    m.cc1 = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, 1 - 0.75))
    m.cc2 = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, 1 - 0.6))
    m.cc3 = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0, 1 - 0.5))
    
    # objective
    m.OBJ = pyo.Objective(expr = sum(m.x[i] for i in m.I), sense = pyo.minimize)
    
    
    # constraints
    @m.Constraint(m.S, doc = 'individual chance constraint 1')
    def C1(m,s1,s2):
        return m.omega1[s1] * m.x[1] + m.x[2] >= 7 - m.M * (1- m.y1[s1,s2])
    
    @m.Constraint(m.S, doc = 'individual chance constraint 2')
    def C2(m,s1,s2):
        return m.omega2[s2] * m.x[1] + 3 * m.x[2] >= 12 - m.M * (1 - m.y2[s1,s2])
    
    @m.Constraint(m.S, doc = 'individual chance constraint 3')
    def C3(m,s1,s2):
        return m.omega1[s1] * m.x[1] + m.omega2[s2] * m.x[2] >= 10 - m.M * (1 - m.y3[s1,s2])
    
    
    
    m.cc1_def = pyo.Constraint(expr = m.cc1 == 1 - sum(m.p[s1,s2] * m.y1[s1,s2] for s1,s2 in m.S))
    m.cc2_def = pyo.Constraint(expr = m.cc2 == 1 - sum(m.p[s1,s2] * m.y2[s1,s2] for s1,s2 in m.S))
    m.cc3_def = pyo.Constraint(expr = m.cc3 == 1 - sum(m.p[s1,s2] * m.y3[s1,s2] for s1,s2 in m.S))
    
    
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return m

### inspect solution

First we look at the values of decision variables of the stochastic program with individual chance constraints from abive.

In [ ]:
m = individual_cc(S1,S2)
#m.pprint()
m.x.pprint()

x : Size=2, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  2.25 :  None : False : False : NonNegativeReals
      2 :     0 :   2.5 :  None : False : False : NonNegativeReals


Next we look at the violated constraints/scenarios.

In [ ]:
print('violated scenarios for individual chance constraint 1:')
print([(s1,s2) for s1,s2 in m.S if pyo.value(m.y1[s1,s2]) == 0])


print('violated scenarios for individual chance constraint 2:')
print([(s1,s2) for s1,s2 in m.S if pyo.value(m.y2[s1,s2]) == 0])

print('violated scenarios for individual chance constraint 3:')
print([(s1,s2) for s1,s2 in m.S if pyo.value(m.y3[s1,s2]) == 0])


violated scenarios for individual chance constraint 1:
[(1, 1), (1, 2), (1, 3)]
violated scenarios for individual chance constraint 2:
[(1, 1), (2, 1), (3, 1), (4, 1)]
violated scenarios for individual chance constraint 3:
[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (3, 1)]


In [ ]:
print('risk of individual chance constraint 1: ' + str(pyo.value(m.cc1)))

print('risk of individual chance constraint 2: ' + str(pyo.value(m.cc2)))

print('risk of individual chance constraint 3: ' + str(pyo.value(m.cc3)))

risk of individual chance constraint 1: 0.25
risk of individual chance constraint 2: 0.333333333333333
risk of individual chance constraint 3: 0.5


### evaluating solution


TBD

# joint vs. individual chance constraints

Of course the choice whether a joint or individual chance constraint is used depends on the problem being modeled, but

### Joint Chance-constraint Pros:

- using multiple inequalities to define a system can result in a more robust solution
- helps modeling specific system constraints to ensure they meet a certain satisfaction level with an anticipated probability.

### Joint Chance-constraint Cons:

- solving large systems with multiple constraints can be difficult
    
### Individual Chance-constraint Pros:

- modeling the entire system requires only one inequality
- may help to simplify the problem statement (of the modeled system)

### Individual Chance-constraint Cons:

- solving may be computationally difficult or even impossible
- oversimplification of the model -> may lead to a partial, incomplete solution

# References

- https://optimization.mccormick.northwestern.edu/index.php/Chance-constraint_method
- https://www.gams.com/latest/docs/UG_EMP_SP.html#UG_EMP_ChanceConstraints